<h3> Please email me if you have any questions: shyam.sk1998@gmail.com <h3>

In [18]:
# import the necessary packages
from PIL import Image
import pytesseract
import cv2
import tempfile
import re
import glob
import pandas as pd
import numpy as np
import os

<h4>IMPORTANT : PLEASE SPECIFY THE PATHS FOR THE BELOW FILES</h4>

In [19]:
#specify the paths to the Indian-Male-Names.csv and Indian-Female-Names.csv files below
FEMALE_PATH = r"C:\Users\shyam\Desktop\Cvision.ai\Indian-Female-Names.csv" 
MALE_PATH = r"C:\Users\shyam\Desktop\Cvision.ai\Indian-Male-Names.csv"
#specify the path to the dataset here (make sure only .jpg files are in the folder)
dataset = glob.glob(r"C:\Users\shyam\Desktop\Cvision.ai\dataset\*.jpg")
#specify the path to the tesseract.exe file below
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [20]:
#pre-processing Indian names dataset (both male and female)
df1 = pd.read_csv(MALE_PATH)
df2 = pd.read_csv(FEMALE_PATH)
df1 = df1[df1['name'].notnull()]
df2 = df2[df2['name'].notnull()]
male_names = df1['name'].tolist()
female_names = df2['name'].tolist()

new_male_names = []
for male in male_names:
    name = male.split('@')
    for i in name:
        name1 = str(i).split()
        if len(name1) > 1:
            new_male_names.append(name1[0])
            new_male_names.append(name1[1])
        else:
            new_male_names.append(name1[0])
            
new_male_names = list(filter(lambda a: a.isalpha(), new_male_names))
#removing duplicates from list
new_male_names = list(dict.fromkeys(new_male_names))


new_female_names = []
for female in female_names:
    name = female.split('@')
    for i in name:
        name1 = str(i).split()
        if len(name1) > 1:
            new_female_names.append(name1[0])
            new_female_names.append(name1[1])
        else:
            new_female_names.append(name1[0])
            
new_female_names = list(filter(lambda b: b.isalpha(), new_female_names))
#removing duplicate names from list
new_female_names = list(dict.fromkeys(new_female_names))

#combining male and female names
names = new_male_names + new_female_names
names = list(dict.fromkeys(names))
names = list(filter(lambda x: len(x) > 2, names))

#some additional cleaning
names.remove('maruti')
names.remove('mul')
names.remove('name')
names.remove('used')
names.remove('one')
names.append('meghna')
names.append('anik')
names.append('srishti')
names.append('nayar')
names.append('kumudha')
names.append('aruldas')


In [22]:
#pre-processing image data

#converting to 300 DPI
def set_image_dpi(file_path):
    im = Image.open(file_path)
    length_x, width_y = im.size
    factor = min(1, float(1024.0 / length_x))
    size = int(factor * length_x), int(factor * width_y)
    im_resized = im.resize(size, Image.ANTIALIAS)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
    temp_filename = temp_file.name
    im_resized.save(temp_filename, dpi=(300, 300))
    return temp_filename

#converting image file to a string using tesseract OCR engine
def getString(file_path):
    filename = set_image_dpi(file_path)
    image = cv2.imread(filename)
    #converting to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # write the grayscale image to disk as a temporary file so we can
    # apply OCR to it
    filename = "{}.png".format(os.getpid())
    cv2.imwrite(filename, gray)
    # load the image as a PIL/Pillow image, apply OCR, and then delete
    # the temporary file
    text = pytesseract.image_to_string(Image.open(filename))
    os.remove(filename)
    text = text.split()
    clean_text = []
    for i in text:
        clean_string = re.sub('(?:[^\w/,]|_)+','', i) #regular expression to eliminate special characters except ('/','_')
        clean_text.append(clean_string)
    return clean_text

#making predictions by using pattern matching
def predict(file_path):
    clean_text = getString(file_path)
    VIN_num = ''
    Reg_no = ''
    engine_no = ''
    mfg_date = ''
    reg_date = ''
    daterange = list(range(2000,2021))
    c_no_found = False
    reg_no_found = False
    eng_no_found = False
    reg_date_found = False
    mfg_date_found = False
    for i in clean_text:
        len_str = len(str(i))
        val_found = False
        while not val_found:
            if len_str >= 15 and i.isalnum() and not i.isdigit() and not i.isalpha():
                if not c_no_found:
                    VIN_num = i
                    val_found = True
                    c_no_found = True
                    break
            if len_str >= 9 and i.isalnum() and not i.isdigit() and not i.isalpha():
                if not reg_no_found:
                    Reg_no = i
                    val_found = True
                    reg_no_found = True
                    break
                if reg_no_found and not eng_no_found:
                    engine_no = i
                    eng_no_found = True
                    val_found = True
                    break
            for date in daterange:
                if str(date) in i:
                    if not reg_date_found:
                        reg_date = i
                        date1 = int(date)
                        reg_date_found = True
                        val_found = True
                    elif reg_date_found and not mfg_date_found:
                        mfg_date = i
                        date2 = int(date)
                        mfg_date_found = True
                        val_found = True
            try:
                if date2 - date1 > 0 and not reg_date_found:
                    temp_x = reg_date
                    temp_y = mfg_date
                    mfg_date = temp_x
                    reg_date = temp_y
            except:
                pass
            if not val_found:
                break
    person_name = []
    indices = []
    lower_text = list(map(str.lower, clean_text))
    for name in lower_text:
        if name.isalpha():
            for mf in names:
                if mf == name:
                    index = lower_text.index(name)
                    indices.append(index)       
    indices = indices[:3]
    try:
        if (indices[2] == indices[1]) or (indices[2] - indices[1] > 1):
            indices.pop(2)
        if (indices[1] == indices[0]):
            indices.pop(1)   
    except:
        pass
    for i in indices:
        person_name.append(clean_text[i])
    person_name = ' '.join(person_name)
    op = [Reg_no, VIN_num, person_name, engine_no, reg_date, mfg_date]
    #print(op)
    return op

predictions = []
print("Making Predictions... Please wait!")
for img in dataset:
    pred = predict(img)
    print('..', end = '')
    predictions.append(pred)
predictions = np.array(predictions)
cols = ['Registration No.', 'Chassis No.', 'Name', 'Engine No.', 'Registration Date', 'MFG Date']
rows = []
for row in dataset:
        row = row.split("\\")[-1]
        rows.append(row)
df = pd.DataFrame(predictions, rows, cols)
df.to_excel("OCR_output.xlsx")
print("\nPredictions completed! Results have been saved to OCR_output.xlsx")